# Descriptor-Based ML
An example of approaches used to do machine learning with chemical descriptors. 

We are going to use [Mordred](https://jcheminf.biomedcentral.com/articles/10.1186/s13321-018-0258-y#Sec1), a Python library that wraps RDKit and other codes so that they work better with the Python data and ML libraries.

**ALCF NOTE**: Make sure to change the kernel to "rapids-21.10"

In [ ]:
%matplotlib inline
from matplotlib import pyplot as plt
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LassoCV
from sklearn.metrics import r2_score, mean_absolute_error
from mordred import Calculator, descriptors
from rdkit import Chem
import pandas as pd
import numpy as np

In [ ]:
np.random.seed(128)

## Load in the Dataset
We are going to use a copy of the [QM9 dataset](https://www.nr2_scorecom/articles/sdata201422) for this example

In [ ]:
data = pd.read_json('datasets/qm9.json.gz', lines=True).sample(1024)  # Do not need the full set to make a point.
print(f'Loaded {len(data)} molecules')

Parse as RDKit molecule objects

In [ ]:
data['mol'] = data['smiles_0'].apply(Chem.MolFromSmiles)

## Descriptor Calculators with Modred
The key unit for working with Mordred is the [``Calculator``](http://mordred-descriptor.github.io/documentation/master/mordred.html#mordred.Calculator) object.

In [ ]:
calc = Calculator(descriptors, ignore_3D=True)

Modred has a lot of descriptors, and we are going to use quite a few of them to demonstrate some points about machine learning. 

In [ ]:
print(f'First 10 out of {len(calc.descriptors)} descriptors')
calc.descriptors[:10]

In [ ]:
desc = calc.pandas(data['mol'], nproc=1)

In [ ]:
print(f'Computed {len(desc.columns)} descriptors for {len(desc)} molecules')

## Preprocessing
The main issue before using all of these features for machine learning, especially on small datasets, is the need to perform feature selection.

There are few issues to be concerned about. 

For one, some of the descriptors are the same for all molecules.

In [ ]:
zero_variation = [c for c, s in desc.std().items() if s == 0]

In [ ]:
print(f'{len(zero_variation)} ({len(zero_variation) / len(desc.columns) *100:.1f}%) features are constant for the dataset. Examples: {zero_variation[:5]}')

Many of these features may be improtant for some datasets but not ours. For example, our dataset contains only C, H, N, O and F so the number of boron atoms is not needed.

We can safely drop these columns

In [ ]:
desc.drop(columns=zero_variation, inplace=True)
print(f'New shape: {desc.shape}')

Next, we are going to convert all of the values to numbers. It just makes things easier

In [ ]:
for c in desc.columns:
    desc[c] = pd.to_numeric(desc[c], errors='coerce')

Then, we look for features that have missing values. 

*Dev Note*: We use the [`isnull`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.isnull.html) and [`any`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.any.html) functions from Pandas to check if any rows of each column have a null value. In many cases, you can write complex loops in Pandas as simple efficient statements. So, if you start writing a loop over a column maybe check the Pandas docs for an example before getting too far.

In [ ]:
missing_values = desc.isnull().any()
print(f'Found {missing_values.sum()} columns with missing values')

In [ ]:
desc = desc.loc[:, ~missing_values]  # Gets only columns that do not (~ means not) have missing values
print(f'New shape: {desc.shape}')

## Testings Components a Model Pipeline
The data require a few more transformations before we can use them in a machine learning model

### Adjust Features to the Same Scale
First, we need to scale the data. If we randomly select a few columns, you will see that they vary wildly in values

There are many ways to scale it. We are going to use the [`StandardScaler`](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html) to adjust so that the mean is zero and the standard deviation is 1.

In [ ]:
scaler = StandardScaler()
feats = scaler.fit_transform(desc)

This "scaler" object holds on to the means and scales of each feature so that we can then apply them to new data. You'll see where this is useful later.

In [ ]:
scaler.mean_[:8], scaler.scale_[:8]

### Compress Features
Many of the features in the dataset are strongly correlated with each other. For larger datasets, the subtle differences between different features could be helpful in building models. For smaller datasets, the strong correlations could lead to overfitting (e.g., some algorithms will learn preferrentially from the signals that are overrepresented in the model)

In [ ]:
cross_corr = np.corrcoef(feats.T)  # Feats.T is the transpose, which allows us to easily compute the correlation between descriptors

Note how many of the features are correlated strongly with each other.

There are [numerous ways](https://scikit-learn.org/stable/modules/classes.html#module-sklearn.decomposition) to compress a set of features to a smaller set of the features that contain roughly the same information. We will use [Principle Component Analysis](https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.PCA.html) for its relative simplicity

PCA creates a new set of features that are linear combinations of the descriptors. One of the best things about PCA is that it can also show how how much of the data is explained by the new features.

Note that as we increase the number of features, we start to gradually explain more of the data.

Let's pick 16 features for now to illustrate the new features

In [ ]:
pca = PCA()

pca.n_components = 16
pca_feats = pca.fit_transform(feats)

These are all mutually orthogonal

In [ ]:
cross_corr = np.corrcoef(pca_feats.T)
print(f'The largest correlation between feature 0 and all others is {max(cross_corr[0, 1:]):.2e}')

### Select Features
The last issue is that not all of the features are correlated well with the target variable. 

In [ ]:
feat_score = [np.corrcoef(data['bandgap'], f)[0][1] for f in pca_feats.T]

In [ ]:
fig, ax = plt.subplots()

ax.bar(range(len(feat_score)), feat_score)

ax.set_ylim(-1, 1)
ax.set_ylabel(f'Correlation with Band Gap')
ax.set_xlabel('Feature ID')

This last portion means that we need to consider even more feature selection to select only those features which are relevant to the problem being modeled.

For that, we are going to use [LASSO](https://scikit-learn.org/stable/modules/linear_model.html#lasso) - a linear regression model that automatically includes only the most important features. The penalty used to determine how many features are used, $\alpha$, must be set by experimenting with different values. Scikit-Learn has the [LassoCV](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LassoCV.html#sklearn.linear_model.LassoCV) class that automatically select the best $\alpha$ for you using cross-validation.

In [ ]:
lasso = LassoCV().fit(pca_feats, data['bandgap'])

In [ ]:
fig, ax = plt.subplots()

ax.bar(range(len(feat_score)), lasso.coef_)
ax.set_xlim(ax.get_xlim())
ax.plot(ax.get_xlim(), [0, 0], 'k--', lw=1.)

ax.set_ylabel(f'Correlation with Band Gap')
ax.set_xlabel('Feature ID')

Depending on your configuration for this notebook, you may notice some of the coefficients are 0.

## Putting it Together
Scikit-Learn includes a [Pipeline](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html) function to make it easier to train models that contain multiple steps. It is a fantastic tool to become familiar with because, as we will demonstrate, it makes experimenting with model settings easier.

Step 1: Make a holdout-set. Not that I split both the data and the descriptors at the same time, which means the same rows (i.e., molecules) get partitioned from each array. `train_data[0]` is the same record as `train_desc[0]`

In [ ]:
train_data, test_data, train_desc, test_desc = train_test_split(data, desc, test_size=0.1)

Step 2: Make a hold-out set

In [ ]:
model = Pipeline([
    ('scale', StandardScaler()),
    ('pca', PCA(n_components=16)),
    ('lasso', LassoCV())
])

In [ ]:
test_data = test_data.copy()  # Avoids some pandas issues
test_data['16-ncomp-model'] = model.fit(train_desc, train_data['bandgap']).predict(test_desc)

Now, let's show how to change the PCA from 16 to only 8 features.

In [ ]:
model.set_params(pca__n_components=8)  # Notation is [step name]__[parameter name]

In [ ]:
test_data['8-ncomp-model'] = model.fit(train_desc, train_data['bandgap']).predict(test_desc)

Compare the two to each other

In [ ]:
fig, axs = plt.subplots(1, 2, sharey=True, figsize=(6.5, 3.))

# Plot both models
for ax, col in zip(axs, ['8-ncomp-model', '16-ncomp-model']):
    ax.set_title(f'{col.split("-")[0]}-Parameter Model')
    r2 = r2_score(test_data[col], test_data['bandgap'])
    ax.text(0.02, 0.9, f'$R^2 = {r2:.2f}$', transform=ax.transAxes)
    ax.scatter(test_data[col], test_data['bandgap'])

# Make it look nicer
for ax in axs:
    ax.set_xlabel('$E_g$, ML')
    
    ax.set_ylim(ax.get_ylim())
    ax.set_xlim(ax.get_ylim())
    ax.plot(ax.get_xlim(), ax.get_xlim(), 'k--')
    
axs[0].set_ylabel('$E_g$, True')
fig.tight_layout()

You can see that you get a better model with adding additional parameters.